In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import files
uploaded = files.upload()
from google.colab import files
uploaded = files.upload()

Saving train_ctrUa4K.csv to train_ctrUa4K.csv


Saving test_lAUu6dG.csv to test_lAUu6dG.csv


In [6]:
train = pd.read_csv("train_ctrUa4K.csv")
test  = pd.read_csv("test_lAUu6dG.csv")

print(train.shape)
print(test.shape)

(614, 13)
(367, 12)


In [7]:
test_ids = test['Loan_ID']

In [12]:
train['source'] = 'train'
test['source']  = 'test'

data = pd.concat([train, test], ignore_index=True)

In [13]:
data['TotalIncome'] = data['ApplicantIncome'] + data['CoapplicantIncome']

data['Income_Loan_Ratio'] = data['TotalIncome'] / data['LoanAmount']

data['EMI'] = data['LoanAmount'] / data['Loan_Amount_Term']

data['LoanAmount_log']  = np.log1p(data['LoanAmount'])
data['TotalIncome_log'] = np.log1p(data['TotalIncome'])

In [14]:
cat_cols = [
    'Gender','Married','Dependents',
    'Self_Employed','Credit_History'
]

for col in cat_cols:
    data[col] = data[col].fillna(data[col].mode()[0])

num_cols = ['LoanAmount','Loan_Amount_Term']

for col in num_cols:
    data[col] = data[col].fillna(data[col].median())

In [15]:
data['Dependents'] = data['Dependents'].replace('3+', 3)

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in data.columns:
    if data[col].dtype == 'object' and col not in ['Loan_Status','source']:
        data[col] = le.fit_transform(data[col].astype(str))

In [17]:
train = data[data['source'] == 'train'].copy()
test  = data[data['source'] == 'test'].copy()

In [18]:
train = train[train['Loan_Status'].notna()]

train['Loan_Status'] = train['Loan_Status'].map({'Y':1,'N':0})

In [19]:
y = train['Loan_Status']

X = train.drop(['Loan_Status','Loan_ID','source'], axis=1)
X_test = test.drop(['Loan_ID','Loan_Status','source'], axis=1)

In [20]:
X = X.fillna(0)
X_test = X_test.fillna(0)

In [21]:
X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)

In [22]:
!pip install xgboost

In [23]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

xgb_model.fit(X, y)

xgb_pred = xgb_model.predict(X_test)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:200: UserWarning: [14:19:52] WARNING: /__w/xgboost/xgboost/src/learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [24]:
final_pred = ['Y' if i==1 else 'N' for i in xgb_pred]

In [25]:
submission = pd.DataFrame({
    'Loan_ID': test_ids,
    'Loan_Status': final_pred
})

submission.to_csv("loan_prediction.csv", index=False)

submission.head()


,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,N


In [26]:
from google.colab import files
files.download("loan_prediction.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>